In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import cv2
from ultralytics import YOLO

## Check for GPU availability

In [3]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA RTX A4000


In [4]:
!nvcc --version
print("\npytorch version:", torch.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0

pytorch version: 2.3.0


## Initialize device to GPU/CPU usage

In [5]:
device = None
if(torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("using", device, "device")

using cuda device


## Initialize the YOLOv9 model

In [6]:
# set to desired gpu number
# torch.cuda.set_device(0)

# specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# use gpu for model
model.to(device)
torch.cuda.synchronize()

## Initialize video source

In [7]:
# Define video source
vid_path = "./sources/two_then_one_ppl.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

Video opened successfully!
Video Frame Rate: 30.0 fps


## Define YOLO classes

In [7]:
PERSON = 0
BIKE = 1
CAR = 2
MOTORCYCLE = 3
BUS = 5

## Process video source

In [9]:
ped_times = {}

while vid_cap.isOpened():
	success, frame = vid_cap.read()
	if not success:
		break

	# Perform detection and tracking
	results = model.track(frame, persist=True, conf=0.2, classes=[PERSON, BIKE], iou=0.5, show=False, tracker="bytetrack.yaml")

	# Print frame
	frame_num = int(vid_cap.get(cv2.CAP_PROP_POS_FRAMES))

	# Get annotated frame and display it
	# annotated_frame = results[0].plot()
	# cv2.imshow('YOLOv9 Tracking', annotated_frame)
	for i, r in enumerate(results):
		for index, box in enumerate(r.boxes):
			# id of object
			print(box.id)
			print(box.cls)
			if(box.id and box.cls[0] != None):
				# print(box.id)
				# print(box.cls)
				tracker_id = int(box.id)
				tracker_cls = int(box.cls[0])
				print("class:", tracker_cls)
				print("id", tracker_id)

				obj_idx = "class_" + str(tracker_cls) + "_idx_" + str(tracker_id)
				if(not(obj_idx in ped_times)):
					ped_times[obj_idx] = frame_num / vid_fps
    
    
	print("frame:", frame_num)

torch.cuda.synchronize()
print(ped_times)

	# # Press 'q' to exit
	# if cv2.waitKey(1) & 0xFF == ord('q'):
	# 	break


0: 384x640 2 persons, 156.6ms
Speed: 3.1ms preprocess, 156.6ms inference, 450.7ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 1

0: 384x640 2 persons, 20.9ms
Speed: 1.9ms preprocess, 20.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 2

0: 384x640 2 persons, 21.2ms
Speed: 1.9ms preprocess, 21.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 3

0: 384x640 2 persons, 20.0ms
Speed: 1.8ms preprocess, 20.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
tensor([1.])
tensor([0.])
class: 0
id 1
tensor([2.])
tensor([0.])
class: 0
id 2
frame: 4

0: 384x640 2 persons, 20.9ms
Speed: 2.3ms preprocess, 20.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
tenso

## Process and Display video source

In [8]:
# Define video source
vid_path = "./sources/2024-05-10T08-45-36.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

# Specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# Use gpu for model
model.to(device)
torch.cuda.synchronize()

# Read and process the video
while vid_cap.isOpened():

    # Get the next captured frame
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform detection and tracking on frame
    results = model.track(
        frame,
        persist=True,
        conf=0.2,
        iou=0.5,
        show=False,
        tracker="bytetrack.yaml",
        classes=[PERSON]
    )

    # Get annotated frame and display it
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv9 Tracking', annotated_frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()

Video opened successfully!
Video Frame Rate: 30.0 fps

0: 384x640 (no detections), 158.8ms
Speed: 2.7ms preprocess, 158.8ms inference, 318.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.3ms
Speed: 4.4ms preprocess, 22.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.4ms
Speed: 2.4ms preprocess, 22.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.0ms
Speed: 2.7ms preprocess, 22.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.5ms
Speed: 2.2ms preprocess, 21.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)



QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is not the object's thread (0x2d54cd20).
Cannot move to target thread (0x2cf14d10)

QObject::moveToThread: Current thread (0x2cf14d10) is n

0: 384x640 (no detections), 21.7ms
Speed: 1.9ms preprocess, 21.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.8ms
Speed: 3.0ms preprocess, 20.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.6ms
Speed: 1.9ms preprocess, 20.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 1.9ms preprocess, 20.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.4ms
Speed: 2.0ms preprocess, 20.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.5ms
Speed: 1.9ms preprocess, 20.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.1ms preprocess, 20.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.9ms
Speed: 2.1ms preprocess, 19.9ms in


0: 384x640 (no detections), 20.1ms
Speed: 2.1ms preprocess, 20.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.9ms
Speed: 2.0ms preprocess, 19.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.4ms
Speed: 2.8ms preprocess, 20.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.6ms
Speed: 2.1ms preprocess, 19.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.0ms
Speed: 1.9ms preprocess, 20.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.7ms
Speed: 1.9ms preprocess, 19.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.8ms
Speed: 2.1ms preprocess, 19.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 2.0ms preprocess, 20.1ms i

Speed: 2.2ms preprocess, 25.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 24.3ms
Speed: 3.3ms preprocess, 24.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 24.6ms
Speed: 2.0ms preprocess, 24.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 26.5ms
Speed: 1.9ms preprocess, 26.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 25.6ms
Speed: 2.0ms preprocess, 25.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 24.8ms
Speed: 2.4ms preprocess, 24.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 24.0ms
Speed: 2.3ms preprocess, 24.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 25.2ms
Speed: 1.9ms preprocess, 25.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person,

...


0: 384x640 (no detections), 25.6ms
Speed: 2.4ms preprocess, 25.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.8ms
Speed: 2.1ms preprocess, 24.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 25.2ms
Speed: 2.0ms preprocess, 25.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.2ms
Speed: 2.4ms preprocess, 24.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.5ms
Speed: 2.1ms preprocess, 25.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.2ms
Speed: 2.0ms preprocess, 24.2ms inferenc


0: 384x640 1 person, 24.5ms
Speed: 2.0ms preprocess, 24.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 28.5ms
Speed: 1.9ms preprocess, 28.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 27.7ms
Speed: 1.9ms preprocess, 27.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 26.1ms
Speed: 1.9ms preprocess, 26.1ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 25.1ms
Speed: 1.9ms preprocess, 25.1ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 25.0ms
Speed: 1.9ms preprocess, 25.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 24.0ms
Speed: 6.2ms preprocess, 24.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 29.0ms
Speed: 2.0ms preprocess, 29.0ms inference, 1.5ms postprocess per image at shape (1, 3, 38


0: 384x640 (no detections), 23.4ms
Speed: 2.1ms preprocess, 23.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.1ms
Speed: 2.2ms preprocess, 23.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.4ms
Speed: 2.0ms preprocess, 23.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.3ms
Speed: 2.0ms preprocess, 23.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.4ms
Speed: 2.0ms preprocess, 23.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 34.0ms
Speed: 2.0ms preprocess, 34.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 23.1ms
Speed: 1.9ms preprocess, 23.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 22.7ms
Speed: 2.0ms preprocess, 22.7ms inference, 1.6ms postprocess per image at shape (1


0: 384x640 (no detections), 20.5ms
Speed: 1.9ms preprocess, 20.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.8ms
Speed: 1.9ms preprocess, 20.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.3ms
Speed: 2.4ms preprocess, 21.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.4ms
Speed: 2.0ms preprocess, 21.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.0ms preprocess, 20.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 1.8ms preprocess, 20.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.2ms
Speed: 1.9ms preprocess, 21.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.7ms preprocess, 20.7ms i


0: 384x640 (no detections), 20.3ms
Speed: 2.0ms preprocess, 20.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.3ms
Speed: 2.0ms preprocess, 20.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.5ms
Speed: 1.9ms preprocess, 20.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.3ms
Speed: 1.8ms preprocess, 20.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.8ms
Speed: 2.2ms preprocess, 20.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.4ms
Speed: 2.4ms preprocess, 20.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.1ms
Speed: 2.0ms preprocess, 20.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.6ms
Speed: 1.9ms preprocess, 20.6ms i

## Process and Display video source

In [11]:
# Define video source
vid_path = "./sources/2024-05-10T08-45-36.mp4"
vid_cap = cv2.VideoCapture(vid_path)
vid_fps = vid_cap.get(cv2.CAP_PROP_FPS)

# Check if the video was openened correctly
if not vid_cap.isOpened():
    print("Error opening video file.")
else:
    print("Video opened successfully!")
    print("Video Frame Rate:", str(vid_fps) + " fps")

# Specify use of gpu in model
model = YOLO('yolov9e-seg.pt')

# Use gpu for model
model.to(device)
torch.cuda.synchronize()

# Read and process the video
while vid_cap.isOpened():

    # Get the next captured frame
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform detection and tracking on frame
    results = model.track(
        frame,
        persist=True,
        conf=0.2,
        iou=0.5,
        show=False,
        tracker="bytetrack.yaml",
        classes=[PERSON]
    )

    # Get annotated frame and display it
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv9 Tracking', annotated_frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()

Video opened successfully!
Video Frame Rate: 30.0 fps

0: 384x640 (no detections), 21.6ms
Speed: 2.0ms preprocess, 21.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
torch.cuda.synchronize()
vid_cap.release()
cv2.destroyAllWindows()